# Demonstration of `pdfplumber`'s table-extraction options

This notebook uses a [report from the FBI's National Instant Criminal Background Check System](../pdfs/background-checks.pdf).

### Import `pdfplumber`

In [ ]:
import pdfplumber
print(pdfplumber.__version__)

### Load the PDF

In [ ]:
pdf = pdfplumber.open("background-checks.pdf")

### Get the first page

In [ ]:
p0 = pdf.pages[1]

In [ ]:
im = p0.to_image()
im

### What data would we get if we used the default settings?

We can check by using `PageImage.debug_tablefinder()`:

In [ ]:
im.reset().debug_tablefinder()

The default settings correctly identify the table's vertical demarcations, but don't capture the horizontal demarcations between each group of five states/territories. So:

### Using custom `.extract_table`'s settings

- Because the columns are separated by lines, we use `vertical_strategy="lines"`
- Because the rows are, primarily, separated by gutters between the text, we use `horizontal_strategy="text"`
- To snap together a handful of the gutters at the top which aren't fully flush with one another, we use `snap_y_tolerance`, which snaps horizontal lines within a certain distance to the same vertical alignment.
- And because the left and right-hand extremities of the text aren't quite flush with the vertical lines, we use `"intersection_tolerance": 15`

In [ ]:
table_settings = {
    "text_x_tolerance": 2,
    "vertical_strategy": "lines",
    "horizontal_strategy": "text",
    "snap_y_tolerance": 0,
    "intersection_x_tolerance": 15,
}
display(im.reset().debug_tablefinder(table_settings))
table = p0.extract_table(table_settings)
for row in table:
    print(row)


### Cleaning up the data

`.extract_table` worked with our custom settings, but the table it detected contains extraneous headers and footers. Since we know that the `Alabama` row is the first, and that there are 56 rows we care about (50 states + DC + 4 territories + the "Totals" row), we can slice away the rest:

In [ ]:
core_table = table[4:4+56]

The first row:

In [ ]:
" • ".join(core_table[0])

The last:

In [ ]:
" • ".join(core_table[-1])

Now, let's turn those rows into dictionaries, and also convert strings-representing-numbers to the numbers themselves, e.g., `"18,870" -> 18870`:

In [ ]:
COLUMNS = [
    "state",
    "permit",
    "handgun",
    "long_gun",
    "other",
    "multiple",
    "admin",
    "prepawn_handgun",
    "prepawn_long_gun",
    "prepawn_other",
    "redemption_handgun",
    "redemption_long_gun",
    "redemption_other",
    "returned_handgun",
    "returned_long_gun",
    "returned_other",
    "rentals_handgun",
    "rentals_long_gun",
    "private_sale_handgun",
    "private_sale_long_gun",
    "private_sale_other",
    "return_to_seller_handgun",
    "return_to_seller_long_gun",
    "return_to_seller_other",
    "totals"
]

In [ ]:
def parse_value(i, x):
    if i == 0: return x
    if x == "": return None
    return int(x.replace(",", ""))

In [ ]:
from collections import OrderedDict
def parse_row(row):
    return {COLUMNS[i]:parse_value(i, cell)
        for i, cell in enumerate(row)}

In [ ]:
data = [ parse_row(row) for row in core_table ]

Now here's the first row, parsed:

In [ ]:
data[0]

### Sort the data

For demonstration purposes, let's list the rows with the highest number of handgun-only background checks:

In [ ]:
for row in list(reversed(sorted(data, key=lambda x: x["handgun"])))[:6]:
    print("{state}: {handgun:,d} handgun-only checks".format(**row))

### Use `extract_text` to extract the report month

It looks like the month of the report is listed in an area 35px to 65px from the top of the page. But there's also some other text directly above and below it. So when we crop for that area, we'll use `.within_bbox` instead of `.crop` to select only characters (and other objects) that are fully within the bounding box.

In [ ]:
month_crop = p0.within_bbox((0, 35, p0.width, 65))
month_crop.to_image()

In [ ]:
month_chars = month_crop.extract_text()
month_chars

---

---

---